# Example based on Excel book

The purpose of this notebook is to study "Modeling Structured Finance Cash Flows with Microsoft Excel: A Step-by-Step Guide" written by Keith A. Allman and to convert what is demonstrated there into Python. Excel is very popular and has many great uses, but this exercise aims to build upon some advantages of Python. One key advantage is access to libraries like QuantLib that allow for calibration of equity and interest rate models, scenario generation, and construction and pricing of financial instruments. This exercise uses QuantLib for Python and stops short of using SWIG to expose functionality that is in QuantLib C++ but not yet Python.

In [22]:
from QuantLib import *
import pandas as pd
from loans import Loan, Loans
from inputs import StructuralInputs
from utils import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from IPython.core.interactiveshell import InteractiveShell #this and the next line produce "pretty" pandas tables
InteractiveShell.ast_node_interactivity = "all"

# Assumptions and Rates
The model will need assumptions for prepayment, defaults (magnitude and timing), and recoveries. It also needs interest rate curves for projecting interest payments and swap legs as well discounting. Finally, it will require some parameters for fees, reserve accounts, and treatment of spread.

In [23]:
sample_rates_and_prepay = pd.read_excel('Sample_Vectors.xlsx', sheet_name='RatesAndPrepay', index_col=0)
sample_loss_timing = pd.read_excel('Sample_Vectors.xlsx', sheet_name='LossTiming', index_col=0)
sample_rates_and_prepay.head()
sample_loss_timing.head()
structural_inputs = StructuralInputs(.02, .01, False, 3, .05, True)
print(structural_inputs)

,1-Month Libor,Custom Rates 1,SMM 1,SMM 2,SDA 50Pct,SDA 100Pct,SDA 200Pct
Month,,,,,,,
1,0.026000,0.04,0.000167,0.01,0.000008,0.000017,0.000033
2,0.028119,0.04,0.000334,0.01,0.000017,0.000033,0.000067
3,0.028744,0.04,0.000501,0.01,0.000025,0.000050,0.000100
4,0.029369,0.04,0.000669,0.01,0.000033,0.000067,0.000133
5,0.029994,0.04,0.000837,0.01,0.000042,0.000083,0.000167


,MonthEnd,Loss Timing 1,Loss Timing 2,Loss Timing 3,Loss Timing 4,Loss Timing 5
MonthStart,,,,,,
1,12,0.033333,0.05,0.2,0.01,0.0
13,24,0.033333,0.05,0.2,0.01,0.0
25,36,0.033333,0.05,0.2,0.01,0.0
37,48,0.033333,0.05,0.2,0.01,0.0
49,60,0.033333,0.05,0.2,0.01,0.0



asset_based fees: 0.02, reserve account percentage: 0.01, capture all excess spread: False,
post default trigger month: 3, default trigger percentage: 0.05, swap active: True


In [2]:
#Construct a yield curve
calc_date = Date(1,2,2007)
Settings.instance().evaluationDate=calc_date
#spot_dates = [Date(1,2,2007), Date(1,3,2017), Date(1,3,2040)]

#spot_rates = [0.09, .09, .09]
day_count = Actual360()
calendar = UnitedStates()
interpolation = Linear()
compounding = Compounded
compounding_frequency = Annual
spot_curve = ZeroCurve(spot_dates, spot_rates, day_count, calendar, interpolation, compounding,
                        compounding_frequency)
spot_curve_handle = YieldTermStructureHandle(spot_curve)
#print(spot_curve_handle.discount(Date(1,3,2017)))
bond_engine = DiscountingBondEngine(spot_curve_handle)

In [25]:
#Create an asset
issue_date = Date(1,2,2007)
stub_date = Date(1,3,2007)
maturity_date = Date(1,2,2037)
rate_type = 'FixedRateBond'
#rate_type = 'FixedRateBond'
amortizing_type = 'FixedAmortizingBond'
tenor = Period(Monthly)
calendar = UnitedStates()
business_convention = Following
termination_business_convention = Following
date_generation = DateGeneration.Forward
end_of_month = False
settlement_days = 0
day_count = Actual360()
#rate = sample_rates_and_prepay['1-Month Libor']
rate = .09
default_rate = .00003*12
 
loan1 = Loan(rate_type, amortizing_type, 100000000.00, 100000000.00, issue_date, maturity_date, stub_date, tenor,
              calendar, business_convention, termination_business_convention, date_generation, end_of_month,
              settlement_days, day_count, rate, sample_rates_and_prepay['SMM 1'], 1, .01, 1,
              sample_loss_timing['Loss Timing 1'], None, .4, Period(5, Months))
 
print(loan1.get_npv(bond_engine))
df = loan1.show_cashflow(False)
#print(loan1.instrument.cashflows())
#print(loan1.get_original_term())
#print(loan1.get_remaining_term(issue_date))
loans = Loans('port1', [loan1])

102057288.91137105


In [26]:
list(df.columns)

['beginning notional balance',
 'notional interest',
 'notional principal amortization',
 'ending notional balance',
 'beginning balance',
 'defaults',
 'prepayments',
 'principal amortization',
 'interest',
 'principal recovery',
 'ending balance',
 'cash available to pay liabilities']

In [31]:
iplot(lines_to_plot(df, ['prepayments','defaults','interest', 'principal amortization', 'principal recovery']))

In [28]:
cols = list(df)
#print(cols)
#date_keys = cols[0:1]
#date_format = '{:%Y-%m-%d}'
num_keys = cols
num_format = '{:,.0f}'
#keys = date_keys+num_keys
keys = num_keys
#formats = [date_format]+[num_format]*len(num_keys)
formats = [num_format]*len(num_keys)
format_dict = dict(zip(keys, formats))
df.style.format(format_dict)

,beginning notional balance,notional interest,notional principal amortization,ending notional balance,beginning balance,defaults,prepayments,principal amortization,interest,principal recovery,ending balance,cash available to pay liabilities
date,,,,,,,,,,,,
2007-02-01,0,0,0,"100,000,000",0,0,0,0,0,0,"100,000,000",0
2007-03-01,"100,000,000","750,000","54,623","99,945,377","100,000,000","2,778","16,673","54,621","749,979",0,"99,925,928","821,273"
2007-04-02,"99,945,377","749,590","55,032","99,890,345","99,925,928","2,778","33,351","55,020","749,424",0,"99,834,779","837,795"
2007-05-01,"99,890,345","749,178","55,445","99,834,900","99,834,779","2,778","50,027","55,413","748,740",0,"99,726,561","854,180"
2007-06-01,"99,834,900","748,762","55,861","99,779,039","99,726,561","2,778","66,692","55,799","747,928",0,"99,601,293","870,419"
2007-07-02,"99,779,039","748,343","56,280","99,722,759","99,601,293","2,778","83,337","56,178","746,989",0,"99,459,000","886,504"
2007-08-01,"99,722,759","747,921","56,702","99,666,057","99,459,000","2,778","99,953","56,550","745,922","1,111","99,299,718","903,537"
2007-09-04,"99,666,057","747,495","57,127","99,608,930","99,299,718","2,778","116,533","56,916","744,727","1,111","99,123,492","919,287"
2007-10-01,"99,608,930","747,067","57,556","99,551,375","99,123,492","2,778","133,067","57,274","743,405","1,111","98,930,374","934,857"
